In [1]:
from random import randint
import numpy as np
import cv2 as cv
import shutil
import os

In [2]:
INPUT_DIRPATH = "data\\1.blueprints"
OUTPUT_DIRPATH = "data\\2.augmented"

IMAGES_PER_CLASS = 1000

ROTATION_RANGE = 10
XY_OFFSET_RANGE = 15
CROP_AMOUNT = 150

NOISE_BRIGHTNESS_AMOUNT = 10
NOISE_COLOR_AMOUNT = 5

In [3]:
def rotate_image(img, angle):
    """Rotate the image"""
    
    img_center = tuple(np.array(img.shape[1::-1]) / 2)
    rot_mat = cv.getRotationMatrix2D(img_center, angle, 1.0)

    return cv.warpAffine(img, rot_mat, img.shape[1::-1], flags=cv.INTER_LINEAR)


def crop_image(img, crop_amount, x, y):
    """Crop and offset the image"""

    x0 = crop_amount + x
    x1 = img.shape[0] - crop_amount + x

    y0 = crop_amount + y
    y1 = img.shape[1] - crop_amount + y

    return img[x0:x1, y0:y1]


def add_noise(img, brightness_amount, color_amount):
    """Add random noise to each pixel"""

    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            rnd_brightness = randint(-brightness_amount, brightness_amount)

            for k in range(img.shape[2]):
                rnd_color = randint(-color_amount, color_amount)

                if img[i][j][k] + rnd_color + rnd_brightness > 255:
                    img[i][j][k] = 255

                elif img[i][j][k] + rnd_color + rnd_brightness < 0:
                    img[i][j][k] = 0

                else:
                    img[i][j][k] += rnd_color + rnd_brightness

    return img

In [4]:
for entry in os.scandir(INPUT_DIRPATH):
    if entry.is_file():
        # Delete output directories if they already exist
        out_dir_name = os.path.splitext(os.path.basename(entry.path))[0]
        out_dir_path = os.path.join(OUTPUT_DIRPATH, out_dir_name)

        if os.path.exists(out_dir_path):
            shutil.rmtree(out_dir_path)

        # Create output directories
        os.makedirs(out_dir_path, exist_ok=True)
        print(os.path.basename(out_dir_path))

        # Read blueprint image
        src_img = cv.imread(entry.path)

        # Render blueprint with random modifications
        for i in range(IMAGES_PER_CLASS):
            out_file_path = os.path.join(out_dir_path, f"{i}.png")

            img = rotate_image(src_img, randint(-ROTATION_RANGE, ROTATION_RANGE))
            img = crop_image(
                img,
                CROP_AMOUNT,
                randint(-XY_OFFSET_RANGE, XY_OFFSET_RANGE),
                randint(-XY_OFFSET_RANGE, XY_OFFSET_RANGE),
            )
            img = cv.resize(img, (16, 16))
            img = add_noise(img, NOISE_BRIGHTNESS_AMOUNT, NOISE_COLOR_AMOUNT)

            cv.imwrite(out_file_path, img)

blank
corner_left_90
corner_right_90
